In [1]:
%matplotlib inline
import openpyxl
#from astropy.table import Table
from astropy.table import vstack, Table
import astropy.io.fits as fits
import matplotlib.pyplot as plt
import numpy as np
import egs_morph
import time
import os
import astropy.stats as stats

import warnings
warnings.filterwarnings("ignore")

In [2]:
egs_path = '/home/hqzhang/Data_EGS/EGS_tot_v1/science_images/'  #22103
psf_path = "psf/"
type_g = 'disk'
fig_path = '/home/wangjh/Mywork/EGS/EGS_figure/%s/'%type_g
result_path = '/home/wangjh/Mywork/EGS/EGS_result/%s/'%type_g
if not os.path.exists(result_path):
    os.makedirs(result_path)
psf_v = fits.open(psf_path + "epsf_v.fits")[0].data
psf_i = fits.open(psf_path + "epsf_i.fits")[0].data
psf_j = fits.open(psf_path + "epsf_j.fits")[0].data
psf_h = fits.open(psf_path + "epsf_h.fits")[0].data

ws = openpyxl.load_workbook('egs_type/egs_%s.xlsx'%type_g)['Sheet1']

begin = 2
#end = 2353 #sph
end = 3400 #disk6815
#end = 814 #irr

In [3]:
failed_list = []
Rp_5_list = []
no_list = []

In [4]:
def psf_final(band):
    if band == 'v':
        psf = psf_v
    elif band == 'i':
        psf = psf_i
    elif band == 'j':
        psf = psf_j
    elif band == 'h':
        psf = psf_h
    return psf

In [5]:
bands = ['v','i','j','h']
table = Table(names=('id','band','redshift','mass','sfr','p_sph','p_disk','p_irr','gal_ser_n_h','Sersic_n','flag_n','snr',\
                       'r20(kpc)','r50(kpc)','re(kpc)','r80(kpc)','r_petro','ellip','C','A','S','Gini','M20','M20_c'),\
                       dtype=('S','S','f2','f2','f4','f2','f2','f2','f2','f2','I',\
                              'f4','f4','f4','f4','f4','f4','f4','f4','f4','f4','f4','f4','f4'))
for i in range(begin,end+1):
    start = time.time()
    id_name = ws.cell(row=i, column=1).value
    print('%s is start _%d'%(id_name,i))
    redshift = ws.cell(row=i, column=6).value    
    mass = ws.cell(row=i, column=7).value
    sfr = ws.cell(row=i, column=8).value
    gilfit_n = ws.cell(row=i, column=9).value
    type_sph = ws.cell(row=i, column=10).value
    type_disk = ws.cell(row=i, column=11).value
    type_irr = ws.cell(row=i, column=12).value
    
    
    img_path = egs_path + '%s/'%id_name
    if os.path.exists(img_path):
        mask = fits.open(egs_path + '%s/%s_mask.fits'%(id_name,id_name))[0].data
        for band in bands:
            image_data = fits.open(egs_path + '%s/%s_%s.fits'%(id_name,id_name,band))[0].data

            psf = psf_final(band)
            image_morph = egs_morph.mymorph(id_name,image_data,psf,mask,redshift ,mass,sfr,band,type_sph,type_disk,type_irr,gilfit_n) 
            bbb = image_morph.save_fig
            if bbb!= 0:        
                bbb.savefig(fig_path + '%s_%s.png'%(id_name,band))
            image_tabel = image_morph.image_table
            table = vstack([table,image_tabel]) 

            if image_morph.worked == 0:
                failed_list.append('%d_%s'%(id_name,band))
            else:
                if image_morph.Rp5 == 0:
                    Rp_5_list.append('%d_%s'%(id_name,band))
        print('%s is done'%id_name,'Time: %g s.' % (time.time() - start))
    else:       
        no_list.append('%d'%id_name)
        print('%s is not exsit'%id_name)
        pass

26 is start _2
26 is done Time: 3.1569 s.
33 is start _3
33 is done Time: 3.21609 s.
37 is start _4
37 is done Time: 2.589 s.
50 is start _5
50 is done Time: 2.64801 s.
51 is start _6
51 is done Time: 3.09448 s.
59 is start _7
59 is done Time: 3.18477 s.
60 is start _8
60 is done Time: 3.34477 s.
65 is start _9
65 is done Time: 2.97874 s.
67 is start _10
67 is done Time: 3.52971 s.
72 is start _11
72 is done Time: 4.0795 s.
74 is start _12
74 is done Time: 2.78371 s.
75 is start _13
75 is done Time: 2.93126 s.
91 is start _14
91 is done Time: 3.3061 s.
97 is start _15
97 is done Time: 2.62439 s.
99 is start _16
99 is done Time: 2.37957 s.
103 is start _17
103 is done Time: 2.14094 s.
107 is start _18
107_2.48 is failed
107 is done Time: 2.54376 s.
112 is start _19
112_2.32 is failed
112 is done Time: 3.53611 s.
118 is start _20
118 is done Time: 2.96223 s.
120 is start _21
120 is done Time: 3.09815 s.
124 is start _22
124 is done Time: 2.95973 s.
131 is start _23
131 is done Time: 2.81

In [6]:
#table.show_in_notebook()

In [7]:
table.write(result_path +'%s1_result.fits'%type_g, overwrite=True)

In [8]:
with open(result_path + "failed_%s_%d-%d.txt"%(type_g,begin,end),'w') as failed_l:
    [failed_l.write(str(item)+'\n') for item in failed_list]
    failed_l.close    

In [9]:
with open(result_path + "Rp5_%s_%d-%d.txt"%(type_g,begin,end),'w') as Rp5_l:
    [Rp5_l.write(str(item)+'\n') for item in Rp_5_list]
    Rp5_l.close    

In [10]:
with open(result_path + "no_%s_%d-%d.txt"%(type_g,begin,end),'w') as no_l:
    [no_l.write(str(item)+'\n') for item in no_list]
    no_l.close    

In [11]:
print('done')

done
